## Mount to Google Drive Folder
To access the dataset and relevant code, we mount our current directory to the desired folder. In this case, it's the project folder "Contrast-GMMs" in which I've backed uo the dataset files. I've placed a shortcut for this folder in my "Projects" folder. 


In [1]:
# This mounts your Google Drive to the Colab VM.
# ------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import os
import sys
%cd 'drive/My Drive/Projects/Contrast-GMMs'

Mounted at /content/drive
/content/drive/Othercomputers/My MacBook Air/Contrast-GMMs


# Import Necessary Libraries 
The following imports are necessary to run the subsequent code

In [2]:
# Imports for our code
# -----------------------------------
from model.NovelNetwork import NovelNetwork
from model.layers.SupConLoss import SupConLoss
from model.layers.MVNet import MV_CNN
import data.utils

# inmports for external libraries
# -----------------------------------
import torch
from collections import OrderedDict

# Model Training with Hyperparameters 


In [3]:
# CONSTANTS
# ---------------------------
BATCH_SIZE = 64
KKC = ["airplane", "toilet", "guitar"]  # Classes seen during training and test time (KKC)
UUC = ["car"]                           # Classes not seen during training time (UUC)
ALL_CLASSES = KKC + UUC                 # All classes
PERC_VAL = 0.20                         # Percent of data for validation 

LATENT_DIMS=3                           # Size of the Latent-Space (Representation is LATENT_DIM x 1)
# ---------------------------

# hyperparameters for our model
args = {
    'print_every' : 100,
    'feat_layer'  : 'fc',
    'feat_sample' : 200,
    'dist_metric' : 'mahalanobis',
    'min_g' : 2,
    'max_g' : 20,
    'epoch' : 15,
    'lr' : 5e-4
}

# Set the device (CUDA compatibility needs to be added to NovelNetwork.py)
USE_GPU = True
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [ ]:
# COLLECTING DATASET
# ---------------------------

print("Collecting ModelNet data . . .\n")
MODELNET = data.utils.get_ModelNet(KKC=KKC, ALL=ALL_CLASSES, BATCH_SIZE=BATCH_SIZE)
print(". . . done!")

# Show examples from MNIST
data.utils.showImg2d(MODELNET['TRAIN'])
# Define the layers
layers = MV_CNN(LATENT_DIMS)


['./data/modelnet40v2/person', './data/modelnet40v2/stool', './data/modelnet40v2/range_hood', './data/modelnet40v2/mantel', './data/modelnet40v2/xbox', './data/modelnet40v2/curtain', './data/modelnet40v2/plant', './data/modelnet40v2/desk', './data/modelnet40v2/tent', './data/modelnet40v2/vase', './data/modelnet40v2/glass_box', './data/modelnet40v2/tv_stand', './data/modelnet40v2/bottle', './data/modelnet40v2/table', './data/modelnet40v2/airplane', './data/modelnet40v2/bowl', './data/modelnet40v2/door', './data/modelnet40v2/bookshelf', './data/modelnet40v2/toilet', './data/modelnet40v2/dresser', './data/modelnet40v2/night_stand', './data/modelnet40v2/laptop', './data/modelnet40v2/bathtub', './data/modelnet40v2/guitar', './data/modelnet40v2/stairs', './data/modelnet40v2/keyboard', './data/modelnet40v2/sink', './data/modelnet40v2/chair', './data/modelnet40v2/cone', './data/modelnet40v2/flower_pot', './data/modelnet40v2/car', './data/modelnet40v2/bed', './data/modelnet40v2/monitor', './da

KeyboardInterrupt: ignored

In [ ]:
 # Run the model
# ------------------------------------------------
new_model = NovelNetwork(layers, known_labels=[i + 1 for i in range(0, len(KKC))], criterion=SupConLoss)
new_model.train(MODELNET['TRAIN'], MODELNET['VAL'], args, print_info=True)
# ------------------------------------------------